### Transform Payments Data
- 1.Extract Date and time from payment_timestamp and create new columns payment_date and payment_time
- 2.Map Payment_status to conatin descriptive values (1-success, 2-Pending, 3-Cancelled, 4-Failed)
- 3.Write transformed data to the silver schema


In [0]:
df = spark.table("gizmobox.bronze.py_payments")
display(df)

1.Extract Date and time from payment_timestamp and create new columns payment_date and payment_time

In [0]:
from pyspark.sql import functions as F
df_extracted_payments=(
    df.select(
        'payment_id',
        'order_id',
        F.date_format('payment_timestamp','yyyy-MM-dd').cast('date').alias('payment_date'),
         F.date_format('payment_timestamp','HH:mm:ss').alias('payment_time'),
         'payment_status',
         'payment_method'


    )
)
display(df_extracted_payments)

2.Map Payment_status to conatin descriptive values (1-success, 2-Pending, 3-Cancelled, 4-Failed)

In [0]:
from pyspark.sql import functions as F
df_final = (df_extracted_payments.
select(
    'payment_id',
    'order_id',
    'payment_date',
    'payment_time',
    F.when(df_extracted_payments.payment_status == 1,'Success').
    when(df_extracted_payments.payment_status == 2,'Pending').
    when(df_extracted_payments.payment_status == 3,'Cancelled').
    when(df_extracted_payments.payment_status == 4,'Failed').
    alias('payment_status'),
    'payment_method'

))
display(df_final)

3.Write transformed data to the silver schema

In [0]:
df_final.writeTo("gizmobox.silver.py_payments").createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_payments;